# Descripción del proyecto

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.
Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 

Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

# Introducción

En Beta Bank, la retención de clientes es una prioridad crítica debido a una creciente tasa de abandono por parte de los usuarios. Atraer nuevos clientes resulta más costoso que mantener a los actuales, lo que hace esencial identificar y retener a aquellos en riesgo de abandonar el banco. 

Este proyecto tiene como objetivo desarrollar un modelo predictivo para determinar la probabilidad de que un cliente abandone el banco en el futuro cercano. Utilizando datos históricos sobre el comportamiento de los clientes, se busca construir un modelo que maximice el valor F1, con un umbral mínimo de 0.59 para su validación. Además, se evaluará el desempeño del modelo a través de la métrica AUC-ROC, proporcionando una comparación adicional con el valor F1 para asegurar una evaluación exhaustiva del modelo.


# Inicialización

In [102]:
# Cargamos todas las librerias que creemos que vamos a utilizar

import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

## Preparación de los datos

In [103]:
# cargamos el archivo en un dataframe

df = pd.read_csv('/datasets/Churn.csv')

In [104]:
#validamos las dimensiones del dataframe (usamos shape) e imprimimos las primeras filas (usamos head)

print(df.shape)
print(df.head())
print()

#mostramos la informacion del dataframe con el metodo info
df.info()

(10000, 14)
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4

### Correción de los datos

In [105]:
#procedemos a cambiar los nombres de columnas ya que cuentan con mayusculas, no  tiene _ en los nombres de mas de 2 palabras.

# creamos una lista con los nombres correctos
new_columns = ['row_number','customer_id', 'surname', 'credit_score', 
               'geography', 'gender', 'age', 'tenure', 'balance', 'num_of_products', 
               'has_cr_card', 'is_active_member', 'estimated_salary', 'exited']

# Asignamos los nuevos nombres de columnas
df.columns = new_columns

# mostramos nuevamente la informacion del dataframe para ver los nuevos nombres de columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [106]:
#validamos el tipo de datos y convertimos la columna 'Tenure' al tipo entero(int64) 
df['tenure'] = df['tenure'].astype('Int64') 

#validamos el cambio del tipo de datos con el metodo info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   Int64  
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: Int64(1), float64(2), int64(8), object(3)
memory usage: 1.1+ MB


In [107]:
# Eliminamos columnas que son irrelevantes para nuestros calculos: row_number, surname
df = df.drop(['row_number', 'surname'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   geography         10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            9091 non-null   Int64  
 6   balance           10000 non-null  float64
 7   num_of_products   10000 non-null  int64  
 8   has_cr_card       10000 non-null  int64  
 9   is_active_member  10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  exited            10000 non-null  int64  
dtypes: Int64(1), float64(2), int64(7), object(2)
memory usage: 947.4+ KB


### Duplicados y valores ausentes

In [108]:
# validamos si tenemos valores duplicados en el dataframe con el metodo duplicated
print(df.duplicated().sum())

0


In [109]:
#validamos si tenemos valores ausentes en el dataframe con el metodo isna
print(df.isna().sum())

customer_id           0
credit_score          0
geography             0
gender                0
age                   0
tenure              909
balance               0
num_of_products       0
has_cr_card           0
is_active_member      0
estimated_salary      0
exited                0
dtype: int64


In [110]:
#Rellamos los valores ausentes de 'tenure' con la mediana, usamos el metodo fillna

tenure_median = df['tenure'].median()
tenure_median = int(tenure_median) #colocamos el valor de la mediana como numero entero.

df['tenure'].fillna(tenure_median, inplace= True)

#validamos nuevamente si tenemos valores ausentes en la columna 'tenure'
print(df['tenure'].isna().sum())

0


### Resumen estadístico de las variables

In [111]:
# Mostramos un resumen estadístico de las variables utilizando el metodo describe
print(df.describe())

        customer_id  credit_score           age       tenure        balance  \
count  1.000000e+04  10000.000000  10000.000000  10000.00000   10000.000000   
mean   1.569094e+07    650.528800     38.921800      4.99790   76485.889288   
std    7.193619e+04     96.653299     10.487806      2.76001   62397.405202   
min    1.556570e+07    350.000000     18.000000      0.00000       0.000000   
25%    1.562853e+07    584.000000     32.000000      3.00000       0.000000   
50%    1.569074e+07    652.000000     37.000000      5.00000   97198.540000   
75%    1.575323e+07    718.000000     44.000000      7.00000  127644.240000   
max    1.581569e+07    850.000000     92.000000     10.00000  250898.090000   

       num_of_products  has_cr_card  is_active_member  estimated_salary  \
count     10000.000000  10000.00000      10000.000000      10000.000000   
mean          1.530200      0.70550          0.515100     100090.239881   
std           0.581654      0.45584          0.499797      5751

### Procesamiento de caracteristicas

In [112]:
#usamos la tecnica One-Hot Encoding para procesar caracteristicas categoricas

df_ohe = pd.get_dummies(df, drop_first= True)
print(df_ohe.head())

   customer_id  credit_score  age  tenure    balance  num_of_products  \
0     15634602           619   42       2       0.00                1   
1     15647311           608   41       1   83807.86                1   
2     15619304           502   42       8  159660.80                3   
3     15701354           699   39       1       0.00                2   
4     15737888           850   43       2  125510.82                1   

   has_cr_card  is_active_member  estimated_salary  exited  geography_Germany  \
0            1                 1         101348.88       1                  0   
1            0                 1         112542.58       0                  0   
2            1                 0         113931.57       1                  0   
3            0                 0          93826.63       0                  0   
4            1                 1          79084.10       0                  0   

   geography_Spain  gender_Male  
0                0            0  
1     

In [113]:
#estandarizamos los datos de las caracteristicas numericas con el escalado de caracteristicas

# definimos las columnas numéricas que necesitan escalado
numerical_columns = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

scaler = StandardScaler()  #creamos la instancia del escalador

# Ajustamos el escalador a los datos y transformar las columnas numéricas
df_ohe[numerical_columns] = scaler.fit_transform(df_ohe[numerical_columns])

# mostramos los datos escalados
print(df_ohe.head())


   customer_id  credit_score       age    tenure   balance  num_of_products  \
0     15634602     -0.326221  0.293517 -1.086246 -1.225848        -0.911583   
1     15647311     -0.440036  0.198164 -1.448581  0.117350        -0.911583   
2     15619304     -1.536794  0.293517  1.087768  1.333053         2.527057   
3     15701354      0.501521  0.007457 -1.448581 -1.225848         0.807737   
4     15737888      2.063884  0.388871 -1.086246  0.785728        -0.911583   

   has_cr_card  is_active_member  estimated_salary  exited  geography_Germany  \
0            1                 1          0.021886       1                  0   
1            0                 1          0.216534       0                  0   
2            1                 0          0.240687       1                  0   
3            0                 0         -0.108918       0                  0   
4            1                 1         -0.365276       0                  0   

   geography_Spain  gender_Male  
0   

Se valida el dataframe mostrando la informacion con el metodo info, en el cual evidenciamos que los tipos de datos de cada columna son correctos, excepto el de la columna 'tenure' el cual se procede a cambiar a tipo de  dato entero; se eliminan las columnas irrelevantes para nuestro analisis como lo son 'row_number' y 'surname'. 
Adicionalmente se revisa si tenemos duplicados lo cual nos da cero y valores ausentes 909, los cuales no se deberian eliminar ya que son datos considerables de clientes, para lo cual se procede a rellenar dichos valores ausentes con la mediana. ademas se realiza un resumen estadistico del dataframe llamando al metodo describe, en el cual podemos ver informacion estadistica como el promedio, valores minimos y maximos, desviacion estandar, etc.

Se procede a usar pd.get_dummies() para procesar caracteristicas categoricas ya que que es más apropiada para características categóricas que no tienen un orden y se estandarizó los datos de las caracteristicas numericas con StandardScaler() para que todas las caracteristicas se consideren igualmente importantes.

## Equilibrio de clases

In [114]:
# Revisamos si las clases en la columna objetivo 'Exited' están equilibradas.
class_frequency = df['exited'].value_counts(normalize =  True)

print(class_frequency)

0    0.7963
1    0.2037
Name: exited, dtype: float64


In [115]:
#separamos las caracteristicas y nuestro objetivo
features = df_ohe.drop('exited', axis=1)
target = df_ohe['exited']

In [116]:
#dividimos los datos en conjunto de entrenamiento, validacion y prueba 

# dividimos el entrenamiento (60%) y un conjunto combinado de validación/prueba (40%)
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.40, random_state=12345)

# ahora dividimos el conjunto combinado (40%) en la mitad para obtener validacion 20% y prueba 20%
features_valid, features_test, target_valid, target_test = train_test_split(features_temp, target_temp, test_size=0.50, random_state=12345)


In [117]:
#entrenamos el modelo
model = RandomForestClassifier(random_state=12345, n_estimators=100)
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid) # Realizamos predicciones en el conjunto de validación

#calculamos el valor de F1 y lo mostramos
print('F1 Score (sin tratar el desequilibrio):', f1_score(target_valid, predicted_valid)) 

F1 Score (sin tratar el desequilibrio): 0.5705705705705705


Obtuvimos del modelo un valor F1 de  0.559 y podriamos decir que esta cercano al solicitado en el proyecto(0.59), teniendo en cuenta que no abordamos el desequilibrio de las clases lo que podria estar afectando a nuestro modelo, este modelo sera nuestra referencia para ver y comparar las mejoras a traves del equilibrio de clases.

## Mejora de la calidad del modelo

### Sobremuestreo

In [118]:
# previamente ya dividimos nuestros datos en entrenamiento y validacion: features_train, features_valid, target_train, target_valid

# Definimos la función de sobremuestreo
def upsample(features, target, repeat):
    features_zeros = features[target == 0]  # Separamos la clase mayor (exited = 0) y la clase menor (exited = 1)
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    # Duplicamos las observaciones de la clase menor
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    # mezclamos el conjunto de datos resultante
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

# Aplicamos el sobremuestreo con una repetición de 10
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)


#### Modelo Bosque aleatorio con el conjunto sobremuestreado

In [142]:
model1 = RandomForestClassifier(random_state=12345, n_estimators=100, class_weight = 'balanced', max_depth= 10)
model1.fit(features_upsampled, target_upsampled)  #entrenamos el modelo

predicted_valid1 = model1.predict(features_valid) # Realizamos predicciones en el conjunto de validación

#calculamos el valor de F1 y lo mostramos
print('F1 Score (Sobremuestreo - Bosque Aleatorio):', f1_score(target_valid, predicted_valid1)) 

F1 Score (Sobremuestreo - Bosque Aleatorio): 0.6300768386388584


#### Modelo Arbol de decisión con el conjunto sobremuestreado

In [144]:
model2 = DecisionTreeClassifier(random_state=12345, class_weight = 'balanced', max_depth= 5)
model2.fit(features_upsampled, target_upsampled)  #entrenamos el modelo

predicted_valid2 = model2.predict(features_valid)  # Realizamos predicciones en el conjunto de validación

#calculamos el valor de F1 y lo mostramos
print('F1 Score (Sobremuestreo - Arbol de decisión):', f1_score(target_valid, predicted_valid2))

F1 Score (Sobremuestreo - Arbol de decisión): 0.5991561181434598


#### Modelo regresion logistica con el conjunto sobremuestreado

In [140]:
model3 = LogisticRegression(random_state=12345, solver='liblinear')
model3.fit(features_upsampled, target_upsampled)  #entrenamos el modelo

predicted_valid3 = model3.predict(features_valid)  # Realizamos predicciones en el conjunto de validación

#calculamos el valor de F1 y lo mostramos
print('F1 Score (Sobremuestreo - regresion logistica):', f1_score(target_valid, predicted_valid3))

F1 Score (Sobremuestreo - regresion logistica): 0.3457402812241522


### Submuestreo

In [122]:
# previamente ya dividimos nuestros datos en entrenamiento y validacion: features_train, features_valid, target_train, target_valid

# Definimos la función de submuestreo
def downsample(features, target, fraction):    
    features_zeros = features[target == 0]  # Separamos la clase mayor (exited = 0) y la clase menor (exited = 1)
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
     
    #usamos la función sample() para eliminar aleatoriamente una proporción de las observaciones de la clase mayor 
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)]
        + [features_ones])
    
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)]
        + [target_ones])
    
    # mezclamos el conjunto de datos resultante
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    

    return features_downsampled, target_downsampled

# Aplicamos el submuestreo con una fraccion de 0.1
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

#### Modelo Bosque aleatorio con el conjunto submuestreado

In [153]:
model4 = RandomForestClassifier(random_state=12345, n_estimators=100, class_weight = 'balanced', max_depth= 3)
model4.fit(features_downsampled, target_downsampled)  #entrenamos el modelo

predicted_valid4 = model4.predict(features_valid) # Realizamos predicciones en el conjunto de validación

#calculamos el valor de F1 y lo mostramos
print('F1 Score (Submuestreo - Bosque Aleatorio):', f1_score(target_valid, predicted_valid4)) 

F1 Score (Submuestreo - Bosque Aleatorio): 0.5740402193784278


#### Modelo Arbol de decision con el conjunto submuestreado

In [162]:
model5 = DecisionTreeClassifier(random_state=12345, class_weight = 'balanced', max_depth= 7)
model5.fit(features_downsampled, target_downsampled)  #entrenamos el modelo

predicted_valid5 = model5.predict(features_valid)  # Realizamos predicciones en el conjunto de validación

#calculamos el valor de F1 y lo mostramos
print('F1 Score (Submuestreo - Arbol de decisión):', f1_score(target_valid, predicted_valid5))

F1 Score (Submuestreo - Arbol de decisión): 0.511744966442953


#### Modelo regresion logistica con el conjunto submuestreado

In [164]:
model6 = LogisticRegression(random_state=12345, solver='liblinear', class_weight = 'balanced')
model6.fit(features_downsampled, target_downsampled)  #entrenamos el modelo

predicted_valid6 = model6.predict(features_valid)  # Realizamos predicciones en el conjunto de validación

#calculamos el valor de F1 y lo mostramos
print('F1 Score (Submuestreo - regresion logistica):', f1_score(target_valid, predicted_valid6))

F1 Score (Submuestreo - regresion logistica): 0.3457402812241522


Se realizaron tecnicas de Sobremuestreo y submuestreo para los diferentes tipos de modelos, como lo son bosque aleatorio, arbol de decision y regresion logistica. Para sobremuestreo el bosque aleatorio arrojo el mas alto valor de F1 siendo 0.630, seguido del arbol de decision en sobremuestreo con un valor de 0.599. para el submuestreo el valor mas alto de F1 lo tiene el modelo de bosque aleatorio (0.57), pero no mas que el valor de sobremuestreo. la regresion logistica en ambas tecnicas nos da un valor de 0.347 siendo el modelo con menor valor de todos los modelos en ambas tecnicas.

Aunque el submuestreo también produce buenos resultados, el sobremuestreo parece ofrecer un F1 score más alto. Esto sugiere que el bosque aleatorio puede beneficiarse más de un conjunto de datos con más ejemplos de la clase minoritaria, como ocurre con el sobremuestreo. por lo anterior se procedera a realizar la prueba final con el modelo de bosque aleatorio sobremuestreado.


## Prueba final

In [169]:
# Entrenamos el modelo de bosque aleatorio sobremuestreado

final_model = RandomForestClassifier(random_state=12345, n_estimators=100, class_weight = 'balanced', max_depth= 10)
final_model.fit(features_upsampled, target_upsampled)  #entrenamos el modelo

predicted_test = final_model.predict(features_test) # Realizamos predicciones en el conjunto de prueba

#calculamos el valor de F1 y lo mostramos
print('F1 Score prueba final:', f1_score(target_test, predicted_test)) 

F1 Score prueba final: 0.6127292340884574


In [170]:
#Validamos los valores de auc_roc

probabilities_test = final_model.predict_proba(features_test) # Calculamos las probabilidades de predicción en el conjunto de prueba

probabilities_one_test = probabilities_test[:, 1] #Extraemos las probabilidades de la clase positiva

auc_roc = roc_auc_score(target_test, probabilities_one_test) # Calculamos el valor de AUC-ROC para las predicciones

print("AUC-ROC Score prueba final:", auc_roc)

AUC-ROC Score prueba final: 0.8558953994402394


# Conclusiones

El valor de F1 0.6127 en el conjunto de prueba es un buen resultado, sobre todo si consideramos que se trata de un problema de clasificación desequilibrado y es ligeramente inferior al obtenido en el conjunto de validación (0.630), lo cual es esperado al usar un conjunto de datos diferente. adicionalmente es adecuada ya que supero el valor 0.59 minimo exigido.

el valor de AUC-ROC de 0.8559 es un excelente resultado, ya que está bastante cerca de 1, lo que significa que el modelo tiene una gran capacidad para distinguir entre las clases (clientes que dejan el banco y los que no lo hacen).

El modelo bosque aleatorio sobremuestreado ha mostrado ser una excelente elección para predecir si un cliente dejara el banco pronto. Con un F1 Score razonable y un AUC-ROC alto, podriamos decir que el modelo está listo para ser utilizado por Betabank

